# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,14.18,86,100,1.06,AU,1707680025
1,1,geraldton,-28.7667,114.6000,22.23,88,0,4.12,AU,1707679892
2,2,college,64.8569,-147.8028,-5.61,86,100,2.06,US,1707680026
3,3,qaqortoq,60.7167,-46.0333,-16.82,77,99,2.66,GL,1707679935
4,4,tarawa,1.4190,172.9840,28.05,89,75,4.12,KI,1707680027


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
cities_hum = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = 'Humidity' ,
    color = 'City',
    hover_cols = ['City'],
    alpha = 0.5
)

# Display the map
cities_hum

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria
criteria_df = city_data_df[(city_data_df['Max Temp'] < 25) & (city_data_df['Max Temp'] > 10) & (city_data_df['Humidity'] < 30)]

# Drop any rows with null values
my_cities = criteria_df.dropna(how='any')

# Display the filtered DataFrame
my_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,dongola,19.1667,30.4833,18.47,11,0,2.97,SD,1707680034
68,68,ghat,24.9647,10.1728,21.28,11,8,3.67,LY,1707680064
112,112,brak,27.5496,14.2714,21.51,18,83,5.31,LY,1707680088
132,132,turabah,21.2141,41.6331,16.02,27,0,2.35,SA,1707680098
231,231,waddan,29.1614,16.1390,21.70,19,100,2.70,LY,1707680160


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy() 

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ['Hotel Name'] =''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
15,dongola,SD,19.1667,30.4833,11,
68,ghat,LY,24.9647,10.1728,11,
112,brak,LY,27.5496,14.2714,18,
132,turabah,SA,21.2141,41.6331,27,
231,waddan,LY,29.1614,16.1390,19,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
limit = 1



params = {
    'categories':categories,
    'apiKey':geoapify_key,
    limit:limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = my_cities.loc[index, 'Lat']
    longitude = my_cities.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
dongola - nearest hotel: Lord
ghat - nearest hotel: فندق تاسيلي
brak - nearest hotel: فندق براك السياحي
turabah - nearest hotel: No hotel found
waddan - nearest hotel: No hotel found
zinapecuaro - nearest hotel: El paraiso de los avies
adrar - nearest hotel: No hotel found
goundam - nearest hotel: No hotel found
al jadid - nearest hotel: فندق البركولي
mian channun - nearest hotel: Blue Moon Hotel
nguigmi - nearest hotel: Guest PAM
awjilah - nearest hotel: No hotel found
pokhara - nearest hotel: Hotel Annapurna View Sarangkot
fort abbas - nearest hotel: No hotel found
dharur - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
15,dongola,SD,19.1667,30.4833,11,Lord
68,ghat,LY,24.9647,10.1728,11,فندق تاسيلي
112,brak,LY,27.5496,14.2714,18,فندق براك السياحي
132,turabah,SA,21.2141,41.6331,27,No hotel found
231,waddan,LY,29.1614,16.1390,19,No hotel found
299,zinapecuaro,MX,19.8667,-100.8167,19,El paraiso de los avies
310,adrar,MR,20.5022,-10.0711,28,No hotel found
329,goundam,ML,16.4145,-3.6708,8,No hotel found
406,al jadid,LY,27.0500,14.4000,9,فندق البركولي
408,mian channun,PK,30.4400,72.3543,24,Blue Moon Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    scale = 3,
    size = 'Humidity',
    color = 'City',
    hover_cols = ['Hotel Name', 'Country'],
    alpha = 0.5
)

# Display the map
map_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)